# CNNs avec Keras sur des données de paysage

## Vérification de l'utilisation de GPU

Allez dans le menu `Exécution > Modifier le type d'execution` et vérifiez que l'on est bien en Python 3 et que l'accélérateur matériel est configuré sur « GPU ».

In [ ]:
!nvidia-smi

## Téléchargement du dataset Landscape depuis un repo git

In [ ]:
!git clone https://github.com/nzmonzmp/dataset-landscape.git
!ls dataset-landscape
print("***")
!ls -l dataset-landscape/seg_train
print("***")
!ls -l dataset-landscape/seg_pred

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import itertools
import os
import pathlib
import random
import typing

import cv2
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
import sklearn.utils
import sklearn.metrics
import tensorflow.keras as keras

## Préparation des données

Pour charger nos données, nous allons combiner plusieurs libraires : [OpenCV](https://opencv.org/), [NumPy](https://numpy.org/) et [scikit-learn](https://scikit-learn.org/stable/). Ces librairies seront appelées depuis la fonction `get_images`.

Après avoir chargé chaque image, nous allons passer leur canaux en RGB puis les redimensionner à 150x150, enfin, par défaut, nous retournerons un dataset mélangé grâce à [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

*Complétez la fonction `get_images` qui va chercher les images dans `dir_path` contenant un dossier par classe. Chaque dossier de classe contient l'ensemble des images de cette classe. Il vous faut attribuer le label correct à chaque image.*

In [ ]:
label_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]


def get_images(dir_path: pathlib.Path, shuffle: bool = True
              ) -> typing.Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]:
  images = []
  labels = []
  file_paths  = []

  # On itère sur les sous-dossier de la racine : ils correspondent chacun à une
  # classe
  for subdir_path in dir_path.iterdir():

    # Attribuez le bon label en fonction du nom du dossier "labels"
    # Votre code ici
    label = None

    # On ajoute chaque image du label (dossier) courant à notre dataset
    for image_path in subdir_path.iterdir():
      # Utilisation de OpenCV pour charger l'image
      image = cv2.imread(str(image_path))
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      # En entrée d'un CNN, toutes les images doivent faire la même taille
      image = cv2.resize(image, (150, 150))
      images.append(image)
      labels.append(label)
      file_paths.append(image_path)

  # Création des tableaux numpy que l'on va retourner
  images, labels, file_paths = map(numpy.array, [images, labels, file_paths])

  # Mélange de ces tableaux
  if shuffle:
    images, labels, file_paths = sklearn.utils.shuffle(images,
                                                       labels,
                                                       file_paths)
  return images, labels, file_paths


# get_images(pathlib.Path("dataset-landscape") / "seg_train")

## Solution

## Appel à `get_images`

In [ ]:
images, labels, file_paths = get_images(
    pathlib.Path("dataset-landscape") / "seg_train")

In [ ]:
print(f"Forme des images : {images.shape}")
print(f"Forme des labels : {labels.shape}")
print(f"Forme des chemins : {file_paths.shape}")

seaborn.countplot(x=labels)
plt.title("Décomptes des différents labels")
plt.ylabel("Décompte")
plt.xlabel("Label")
plt.show()

In [ ]:
# Création de la grille de sous-plots. On donne l'argument figsize pour agrandir
# la taille de la figure qui est petite par défaut
f, ax = plt.subplots(5, 5, figsize=(15, 15))

# On choisit 25 indices au hasard, sans replacement (on ne veut pas afficher la
# même image deux fois)
random_indexes = numpy.random.choice(images.shape[0],
                                     size=(5, 5),
                                     replace=False)

for i in range(5):
  for j in range(5):
    img_index = random_indexes[i, j]
    image = images[img_index]
    label = label_names[labels[img_index]]

    # Affichage avec matplotlib et sa fonction imshow, très pratique en vision par
    # ordinateur
    ax[i, j].imshow(image)
    ax[i, j].set_title(f"Exemple {img_index} ({label})")
    ax[i, j].axis('off')

## Création du modèle

Voici un exemple de CNN « minimaliste »

In [ ]:
# Initialisation et définition du modéle

# Le modèle est un empilement de couches où le flux de données est séquentiel
# déclarer et créer un modèle sequentiel 
# Votre Code ici


# Une première couche de 1 convolutions de 3x3 pixels
# VOtre code ici

# Une couche de max pooling
# Votre code ici

# Une couche de manipulation des tenseurs : suppression de toutes les dimensions
# sauf celle de batch et une autre qui contient toutes les valeurs (flaten)
# VOtre code ici

# Une couche de sortie dense avec 6 neurones et un softmax comme activation
# VOtre code ici

# Compilation du modèle avec la définition de la fonction de perte
# Votre code ici
# Optimizer :  Adam, learning rate = 0.0001
# loss : sparse categorical crossentropy
# metric : accuracy


# Affichage d'un résumé du modèle
# Votre code ici

## Pouvez-vous expliquer les différents nombres de paramètres ?

### Solution


## Apprentissage

Apprenons ce modèle sur nos données ! Dans un premier temps, nous entraînons sur une seule epoch pour simplement vérifier que notre modèle est opérationnel.

In [ ]:
# Apprentissage du modèle
# Votre code ici

## Améliorez cette performance

Inspirez-vous du modèle précédent en rajoutant des couches, en faisant des couches plus petites ou plus grosses.

Visez entre 10 et 20 itérations et mois de 1 minute par itération (pour des raisons évidentes).

On peut considérer l'utilisation d'une couche de dropout juste avant la dernière couche dense pour améliorer la régularisation.

On peut obtenir une précision supérieure à 70% sur la base de validation en un temps raisonnable.

La solution proposée prend $\approx$ 45 secondes par itération pendant 15 itérations et atteint aux alentour de 85% d'accuracy sur la base de validation.

In [ ]:
# Vos améliorations ici


### Solution

In [ ]:
# Votre code ici

In [ ]:
# Apprentissage du modèle
# # Votre code ici

# Visualisation des métriques d'entrainement
# Votre code ici


## Évaluation des performances sur l'ensemble de test

Dans le dossier `seg_test` se trouve un ensemble de données qui n'ont jamais été vues durant l'apprentissage.

On utilisera la méthode `evaluate(X, y)` du modèle pour évaluer la qualité de nos prédictions sur ce dataset.

In [ ]:
test_images,test_labels, test_file_paths = get_images(
    pathlib.Path("dataset-landscape") / "seg_test")

# Evaluation 
# Votre code ici


## Analyse d'erreur

On affiche la matrice de confusion puis on regarde des images mal classées.

In [ ]:
def analyze_preds(preds, labels):
  confusion_matrix = sklearn.metrics.confusion_matrix(preds,
                                                      labels,
                                                      normalize="true")
  seaborn.heatmap(confusion_matrix,
                  cmap="rocket_r",
                  xticklabels=label_names,
                  yticklabels=label_names)
  plt.title("Matrice de confusion")
  plt.show()

  seaborn.countplot(x=list(map(lambda x: label_names[x], preds)))
  plt.title("Décomptes des classes prédites")
  plt.ylabel("Décompte")
  plt.xlabel("Class")
  plt.show()


test_pred = numpy.argmax(model.predict(test_images), axis=-1)
analyze_preds(test_pred, test_labels)

In [ ]:
def plot_mistakes(predicted_class: str, true_class: str) -> None:
  print(f"Prédiction : {predicted_class}, classe réelle : {true_class}")
  mistakes = test_images[(test_pred == label_to_index[predicted_class])
                         & (test_labels == label_to_index[true_class])]
  random_indexes = numpy.random.choice(mistakes.shape[0],
                                       size=min(mistakes.shape[0], 25),
                                       replace=False)
  grid_indexes = itertools.product(range(5), repeat=2)

  _, ax = plt.subplots(5, 5, figsize=(15, 15))
  for img_index, (i, j) in zip(random_indexes, grid_indexes):
    ax[i, j].imshow(mistakes[img_index])
    ax[i, j].axis("off")
  plt.show()

In [ ]:
# Plot les images prédites glacier alors qu'elles ont un label montagne
plot_mistakes("glacier", "mountain")

In [ ]:
# Plot les images prédites glacier alors qu'elles ont un label mer
plot_mistakes("glacier", "sea")

In [ ]:
# Plot les images prédites bâtiment alors qu'elles ont un label mer
plot_mistakes("buildings", "sea")